In [ ]:
from google.colab import files
uploaded = files.upload()

In [0]:
import tensorflow as tf
import pandas as pd
import io
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from math import sqrt

In [0]:
train = pd.read_csv(io.StringIO(uploaded['winequality-white-training.csv'].decode('utf-8'))).dropna()
scaler = MinMaxScaler()

train_labels = pd.DataFrame()
train_labels['quality'] = train['quality']
train_values = train.drop(['Id', 'quality'], axis=1)

train_val = np.array(train_values.values.tolist(), dtype=float)
train_val = scaler.fit_transform(train_val)
train_lab = np.array(train_labels.values.tolist(), dtype=float)
    
test = pd.read_csv(io.StringIO(uploaded['winequality-white-testing_with_labels.csv'].decode('utf-8'))).dropna()
test_values = np.array(test.drop(['Id','quality'], axis=1).values.tolist(), dtype=float)
    
test_df = pd.DataFrame()
test_df['quality'] = test['quality']
test_labels = np.array(test_df.values.tolist(), dtype=float)
test_values = scaler.fit_transform(test_values)

test1 = pd.read_csv(io.StringIO(uploaded['winequality-white-testing.csv'].decode('utf-8'))).dropna()
test_df = test1.drop(['Id'], axis=1)
test_values1 = np.array(test_df.values.tolist(), dtype=float)
test_values1 = scaler.fit_transform(test_values1)

In [0]:
learning_rate = 0.00001
epochs = 1000000
batch_size = 1
dropout = 0.5

In [0]:
x = tf.placeholder(tf.float32, [None, 11])
y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32) 

W1 = tf.Variable(tf.random_normal([11, 12], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([12]), name='b1')

W2 = tf.Variable(tf.random_normal([12, 6], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([6]), name='b2')

W3 = tf.Variable(tf.random_normal([6, 3], stddev=0.03), name='W3')
b3 = tf.Variable(tf.random_normal([3]), name='b3')

W4 = tf.Variable(tf.random_normal([3, 1], stddev=0.03), name='W4')
b4 = tf.Variable(tf.random_normal([1]), name='b4')

In [0]:
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

hidden_out1 = tf.add(tf.matmul(hidden_out, W2), b2)
hidden_out1 = tf.nn.relu(hidden_out1)

hidden_out2 = tf.add(tf.matmul(hidden_out1, W3), b3)
hidden_out2 = tf.nn.relu(hidden_out2)

y_ = tf.add(tf.matmul(hidden_out2, W4), b4)

square_fn = tf.square(y_ - y)
loss = tf.reduce_sum(square_fn)

optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

init_op = tf.global_variables_initializer()

In [0]:
with tf.Session() as sess:
  sess.run(init_op)
  total_batch = int(len(train_lab) / batch_size)
  for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x = train_val[i*batch_size:(i+1)*batch_size] 
            batch_y = train_lab[i*batch_size:(i+1)*batch_size]
            _, c = sess.run([optimiser, loss], 
                         feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            avg_cost += c / total_batch
        if (epoch+1) % 100 == 0:
            print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost/batch_size))
  predicted = sess.run(y_, feed_dict={x: test_values1})

mse = mse/len(test_labels)
print(sqrt(mse))
full_mat = []
for i in range(len(predicted)):
  full_mat.append([i, predicted[i][0]])

labels = ['Id', 'quality']
new_df = pd.DataFrame.from_records(full_mat, columns=labels)
new_df.to_csv('predictions_wine.csv',index=False)
#print(new_df.head(5))
files.download('predictions_wine.csv')